# Logic Programming in Python

We have been working through a number of examples of logic problems by hand. Such exercises can be fun and rewarding, and are certainly useful for developing our understanding. However, it is clearly not an approach that scales well and we would like to use the computer to solve logic problems.

The history of solving logic problems with the computer has a long and rich history. Noting that the earliest attempts to create AI systems were rooted in logic, and especially in **symbol manipulation**, we can look back on the history of computing and see major advances that were motivated by this problem. For example, the LISP programming language was specifically designed for symbol manipulation and introduced many innovations in programming languages including garbage collection, support for recursion, first-class functions, and proper conditional expressions not requiring a `goto`. It has had major influence on both AI and on programming language design. Prolog embodies the notions and concepts that we are studying here even more explicitly.

But we are not using LISP or Prolog; we are using Python, and Python does not have any in-built symbolic manipulation or logical inference capabilities (it does include simple logical operators but these are of limited utility for our purposes. We must therefore use a library.

I have not found a library that I am particularly satisfied with. The best I have found is `kanren`, and that is what we shall use. It is not the most intuitive of packages, and the documentation has room for improvement though, so we shall have to develop our understanding slowly.

Our first task is to install kanren. The version of kanren we shall use here is 0.2.3 and this can be found on [PyPI](https://pypi.org/project/kanren/). Kanren does not seem to be available in the Anaconda distribution, unfortunately but it is available via Pip. A possible alternative is `minikanren` which is available in Anaconda; however, I was not able to get this to work.

Let's first install the package. Before beginning, you may wish to create a virtual environment for this module which will create a separate environment to keep it cleanly separated from other modules and thus avoid potential library version conflicts.

* Create a folder for the course
* Open a terminal, navigate to the folder, and enter the command
```
python3 -m venv ./myenv
```
This creates a new folder `myenv` (you can call it what you want) that will store a local python installation.
* Activate the local installation with `myenv\bin/activate.bat` (Windows) or `source myenv/bin/activate` (Mac/Linux).
* Any libraries that you install whilst an environment is activated will be installed into the environment only, and not into the global environment.
* When you are finished, you can deactivate the environment with `deactivate`.

Now you have an environment for the module, we can install libraries into it. For now, we only need the `kanren` module

In [2]:
!pip3 install kanren

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


## Solving basic logic problems in Kanren

One of the most basic notions in kanren is the **logic variable**. These can be created very easily. The following cell imports the command for creating a logic variable and create a new logic variable called `x`.

In [3]:
from kanren import var
x = var()

This code doesn't do anything tangible, but it does set up the machinery that we need to solve problems. Here is one of the very simplest problems we can solve using logic programming:
* Find one number $x$ such that $x==5$.

In [4]:
from kanren import eq, run
run(1, x, eq(x,5))

(5,)

In Kanren, no computations are done until the `run` function is called. `run` takes three or more arguments:
* The number of results to return (0: all of them), in this example there is only one result.
* The logic variable for which we are trying to solve (`x` in this case)
* One or more **goals**. `eq` is an example of a *goal constructor*.

Here is a more complex example that uses two logic variables and two goals. Can you understand what this code does and how it does it?

In [5]:
from kanren import vars
y, z = vars(2)
run(1, x, eq(x,z), eq(z,7))

(7,)

Another common goal constructor is `membero` which looks to see whether an item belongs to some collection. For example, the following example returns one (1) member `x` from the collection `mylist`:

In [6]:
from kanren import membero
mylist = [1,2,3,2,4,2,5]
run(1,x,membero(x,mylist))


(1,)

We can modify this easily to return two members of the collection:

In [7]:
run(2,x,membero(x,mylist))

(1, 2)

or three members of the collection:

In [8]:
run(3,x,membero(x,mylist))

(1, 2, 3)

or **all** members of the collection

In [9]:
run(0,x,membero(x,mylist))

(1, 2, 3, 4, 5)

## Relationships in Kanren

Kanren supports the definition of *relations* between things, and inference over those relations. 

In [10]:
from kanren import Relation, facts
bigger = Relation()
facts(bigger, ("Elephant", "Hippopotamus"),
      ("Hippopotamus", "Horse"),
      ("Elephant", "Horse"),
      ("Horse", "Dog"))

In terms of the logic operators we have been studying, a `Relation` in Kanren is essentially a material implication. A Kanren relation `bigger("Elephant", "Horse")` is roughly equivalent to 
`if Elephant(x) then x is bigger than Horse` and `if Horse(x) then Elephant is bigger than x`

In [11]:
run(1, x, bigger(x,"Hippopotamus"))

('Elephant',)

In [12]:
run(2, x, bigger(x,"Horse"))

('Hippopotamus', 'Elephant')

However, the rules aren't chained together so when we ask for animals bigger than a dog:

In [13]:
run(0, x, bigger(x,"Dog"))

('Horse',)

we only get "Horse". However, as with the earlier examples, we can build compound queries:

In [14]:
run(0, x, bigger(y,"Dog"), bigger(x,y))

('Hippopotamus', 'Elephant')

This would allow us to write a recursive query that found all animals bigger than a dog:

## Other types of Goal Constructor
There are several other type of goal constructor available
* `lall` means that all specified goals have to be True (logical AND)
* `lany` means that any of the specified goals have to be True (logical OR)

So to get an animal that is bigger that a Hipppotamus or bigger than a Horse:

In [15]:
from kanren import lany, lall
run(0, x, lany(bigger(x,"Hippopotamus"), bigger(x,"Horse")))

('Elephant', 'Hippopotamus')

An animal that is bigger than an Horse and bigger than a Hippopotamus:

In [16]:
run(0, x, lall(bigger(x,"Hippopotamus"), bigger(x,"Horse")))

('Elephant',)

Finally, if we wanted to find and animal that is bigger than a horse and smaller than elephant, or smaller than a horse we can use `conde`, which provides logical `AND-OR`.

In [17]:
from kanren import conde
run(0, x, conde(
    (bigger(x,"Horse"), bigger("Elephant",x)),
    (bigger("Horse",x),))
   )

('Hippopotamus', 'Dog')

The `help` for `conde` is quite useful here:

In [18]:
help(conde)

Help on function conde in module kanren.core:

conde(*goalseqs)
    Logical cond
    
    Goal constructor to provides logical AND and OR
    
    conde((A, B, C), (D, E)) means (A and B and C) or (D and E)
    Equivalent to the (A, B, C); (D, E) syntax in Prolog.
    
    See Also:
        lall - logical all
        lany - logical any



## Worked Example

Let us now try to use Kanren to solve a Logic problem. We will use one that we have seen before.

Ahmed, Chen, and Niamh are three friends. They went for dinner together one evening. They decided that they wanted to try all of the items on the menu so they all chose a different main course and a different dessert. The menu options for the main course were Pizza, Daal, and Falafel. For dessert, the choices were Apple Pie, Cheesecake, and Ice Cream. Given the five statements below, can you work out who ordered what?

1. The person who had the Pizza did not have the Apple Pie.
1. Niamh had the Daal.
1. Ahmed did not have Cheesecake.
1. Chen had the Apple Pie.
1. The person who had the Ice Cream did not have the Falafel.

It isn't immediately obvious how to formulate this so let's begin by making some observations.
* We need all of these to be True so it seems likely that our goal will be constructed using `lall`.
* We have to be able to deal with negations, but **Kanren does not support direct negation** (there are good technical reasons for this). We may need to reformulate the problem to account for this.

Where do we even start? It isn't obvious, so let's define a skeleton that we can work with. We'll create a logical variable called `diners` that we will solve for, and a set of `rules`, all of which must be True

In [48]:
diners = var()

rules = lall()

solution = run(0, diners, rules)

print(solution)


(~_372,)


This will turn out to be all that we need - except for the actual rules themselves. The rules need to define all of our knowledge of the solution. The logic solver will find sets of statements for which all of our rules hold.

First, we know that each solution needs to contain logic three variables (name, dinner, dessert). We need to tell the solver this.

In [52]:
diners = var()

rules = lall(
    (eq, (var(), var(), var()), diners)
)

solution = run(0, diners, rules)

print(solution)

((~_458, ~_459, ~_460),)


Notice how this is formatted: the function and its arguments are passed separately. This is because we do not want the `eq` function to be evaluated at the point at which it is defined, Kanren's internals will combine them at the appropriate time.

Let's now add one of the easier clues: "Niamh had the Daal".

In [53]:
diners = var()

rules = lall(
    (eq, (var(), var(), var()), diners),
    (membero, ("Niamh", "Daal", var()), diners)
)

solution = run(0, diners, rules)

print(solution)

((('Niamh', 'Daal', ~_465), ~_463, ~_464), (~_462, ('Niamh', 'Daal', ~_465), ~_464), (~_462, ~_463, ('Niamh', 'Daal', ~_465)))


This line states that an entry in which Niamh had the Daal must be present in the solution. We see that this is indeed the case, but note that kanren has not yet been able to properly construct a solution because we have not yet specified the problem adequately.

Another easy clue is "Chen had the Apple Pie".

In [55]:
diners = var()

rules = lall(
    (eq, (var(), var(), var()), diners),
    (membero, ("Niamh", "Daal", var()), diners),
    (membero, ("Chen", var(), "Apple Pie"), diners)
)

solution = run(0, diners, rules)

for s in solution:
    print(s)

(('Niamh', 'Daal', ~_476), ('Chen', ~_477, 'Apple Pie'), ~_475)
(('Chen', ~_477, 'Apple Pie'), ('Niamh', 'Daal', ~_476), ~_475)
(('Chen', ~_477, 'Apple Pie'), ~_474, ('Niamh', 'Daal', ~_476))
(('Niamh', 'Daal', ~_476), ~_474, ('Chen', ~_477, 'Apple Pie'))
(~_473, ('Niamh', 'Daal', ~_476), ('Chen', ~_477, 'Apple Pie'))
(~_473, ('Chen', ~_477, 'Apple Pie'), ('Niamh', 'Daal', ~_476))


Let's look now at one of the more complex clues which we cannot express quite so easily: "Ahmed did not have Cheesecake". Kanren does not have a `not` operator/goal constructor and so we need to formulate this differently. We can do this by noticing that this rule means that Ahmed must have had a dessert that was not Cheesecake, which means it must have been either Apple Pie or Ice Cream. We can express this using the `lany` goal constructor:

In [ ]:
diners = var()

rules = lall(
    (eq, (var(), var(), var()), diners),
    (membero, ("Niamh", "Daal", var()), diners),
    (membero, ("Chen", var(), "Apple Pie"), diners),
    lany(
        (membero,("Ahmed",var(),"Apple Pie"),diners),
        (membero,("Ahmed",var(),"Ice Cream"),diners)
    )
)

solution = run(0, diners, rules)

for s in solution:
    print(s)

(('Ahmed', ~_484, 'Apple Pie'), ('Niamh', 'Daal', ~_482), ('Chen', ~_483, 'Apple Pie'))
(('Ahmed', ~_485, 'Ice Cream'), ('Niamh', 'Daal', ~_482), ('Chen', ~_483, 'Apple Pie'))
(('Niamh', 'Daal', ~_482), ('Ahmed', ~_484, 'Apple Pie'), ('Chen', ~_483, 'Apple Pie'))
(('Niamh', 'Daal', ~_482), ('Ahmed', ~_485, 'Ice Cream'), ('Chen', ~_483, 'Apple Pie'))
(('Niamh', 'Daal', ~_482), ('Chen', ~_483, 'Apple Pie'), ('Ahmed', ~_484, 'Apple Pie'))
(('Niamh', 'Daal', ~_482), ('Chen', ~_483, 'Apple Pie'), ('Ahmed', ~_485, 'Ice Cream'))
(('Ahmed', ~_484, 'Apple Pie'), ('Chen', ~_483, 'Apple Pie'), ('Niamh', 'Daal', ~_482))
(('Ahmed', ~_485, 'Ice Cream'), ('Chen', ~_483, 'Apple Pie'), ('Niamh', 'Daal', ~_482))
(('Chen', ~_483, 'Apple Pie'), ('Ahmed', ~_484, 'Apple Pie'), ('Niamh', 'Daal', ~_482))
(('Chen', ~_483, 'Apple Pie'), ('Ahmed', ~_485, 'Ice Cream'), ('Niamh', 'Daal', ~_482))
(('Chen', ~_483, 'Apple Pie'), ('Niamh', 'Daal', ~_482), ('Ahmed', ~_484, 'Apple Pie'))
(('Chen', ~_483, 'Apple Pie'), (

We see now that we have a solution of the correct form but with some repetitions because we still have not completely specified the problem.

The other two clues can be expressed in the same way following reframing:
* "The person who had the Pizza did not have the Apple Pie" becomes "The person who had Pizza had Ice Cream or Cheesecake".
* "The person who had the Ice Cream did not have the Falafel" becomes " The person who had Ice Cream had Daal or Pizza".


In [57]:
diners = var()

rules = lall(
    (eq, (var(), var(), var()), diners),
    (membero, ("Niamh", "Daal", var()), diners),
    (membero, ("Chen", var(), "Apple Pie"),diners),
    lany(
        (membero, (var(), "Pizza", "Cheesecake"), diners),
        (membero, (var(), "Pizza", "Ice Cream"), diners)
    ),
    lany(
        (membero,("Ahmed",var(),"Apple Pie"),diners),
        (membero,("Ahmed",var(),"Ice Cream"),diners)
    ),
    lany(
        (membero,(var(),"Falafel","Apple Pie"),diners),
        (membero,(var(),"Falafel","Cheesecake"),diners)
    )
)

solution = run(0, diners, rules)
for s in solution:
    print(s)

(('Ahmed', 'Pizza', 'Ice Cream'), ('Chen', 'Falafel', 'Apple Pie'), ('Niamh', 'Daal', ~_490))
(('Chen', 'Falafel', 'Apple Pie'), ('Ahmed', 'Pizza', 'Ice Cream'), ('Niamh', 'Daal', ~_490))
(('Chen', 'Falafel', 'Apple Pie'), ('Niamh', 'Daal', ~_490), ('Ahmed', 'Pizza', 'Ice Cream'))
(('Ahmed', 'Pizza', 'Ice Cream'), ('Niamh', 'Daal', ~_490), ('Chen', 'Falafel', 'Apple Pie'))
(('Niamh', 'Daal', ~_490), ('Ahmed', 'Pizza', 'Ice Cream'), ('Chen', 'Falafel', 'Apple Pie'))
(('Niamh', 'Daal', ~_490), ('Chen', 'Falafel', 'Apple Pie'), ('Ahmed', 'Pizza', 'Ice Cream'))


We now have an almost complete (albeit repetitive solution), but there is still a gap: what did Niamh have for dessert? Look back at how you solved this problem by hand. How did you deduce that Niamh had the Cheesecake for dessert? We used the constraint that everyone had to have ordered something, and having eliminated all of the other possibilities we deduced that this was the only remaining possibility.

We have not yet encoded that constraint into the rules. Let us do so now. The way we will do this is by adding constaints that every person, main course, and dessert must appear.

In [ ]:
diners = var()

rules = lall(
    (eq, (var(), var(), var()), diners),
    lany(
        (membero, (var(), "Pizza", "Cheesecake"), diners),
        (membero, (var(), "Pizza", "Ice Cream"), diners)
    ),
    (membero, ("Niamh", "Daal", var()), diners),
    lany(
        (membero,("Ahmed",var(),"Apple Pie"),diners),
        (membero,("Ahmed",var(),"Ice Cream"),diners)
    ),
    (membero, ("Chen", var(), "Apple Pie"),diners),
    lany(
        (membero,(var(),"Falafel","Apple Pie"),diners),
        (membero,(var(),"Falafel","Cheesecake"),diners)
    ),
    (membero, ("Ahmed",var(),var()),diners),
    (membero, ("Niamh",var(),var()),diners),
    (membero, ("Chen",var(),var()),diners),
    (membero, (var(),"Falafel",var()),diners),
    (membero, (var(),"Pizza",var()),diners),
    (membero, (var(),"Daal",var()),diners),
    (membero, (var(),var(),"Cheesecake"),diners),
    (membero, (var(),var(),"Apple Pie"),diners),
    (membero, (var(),var(),"Ice Cream"),diners)
)

solution = run(0, diners, rules)
for s in solution:
    print(s)

(('Ahmed', 'Pizza', 'Ice Cream'), ('Chen', 'Falafel', 'Apple Pie'), ('Niamh', 'Daal', 'Cheesecake'))
(('Chen', 'Falafel', 'Apple Pie'), ('Ahmed', 'Pizza', 'Ice Cream'), ('Niamh', 'Daal', 'Cheesecake'))
(('Chen', 'Falafel', 'Apple Pie'), ('Niamh', 'Daal', 'Cheesecake'), ('Ahmed', 'Pizza', 'Ice Cream'))
(('Ahmed', 'Pizza', 'Ice Cream'), ('Niamh', 'Daal', 'Cheesecake'), ('Chen', 'Falafel', 'Apple Pie'))
(('Niamh', 'Daal', 'Cheesecake'), ('Ahmed', 'Pizza', 'Ice Cream'), ('Chen', 'Falafel', 'Apple Pie'))
(('Niamh', 'Daal', 'Cheesecake'), ('Chen', 'Falafel', 'Apple Pie'), ('Ahmed', 'Pizza', 'Ice Cream'))


This now uniquely solves the problem: the solutions are identical but just permuted.

Kanren can be used for solving any general logic problem, but one often has to put considerable thought into how to formulate the problem appropriately. Let's practice this.